<a href="https://colab.research.google.com/github/ankshah131/localsolve-open/blob/main/wildfires/la_wildfires/Vegetation_mapping_with_burn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

In [2]:
ee.Authenticate()

In [3]:
import geemap
import geemap.plot as plot

In [4]:
ee.Initialize(project='ee-agarwalpal2204')

In [5]:
map = geemap.Map()
map.setCenter(-118.288105,34.172376,10)

In [6]:
def mask_clouds(image):
    # Get the cloud mask from the QA60 band
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0)  # Bit 10 is clouds
    return image.updateMask(cloud_mask).divide(10000)  # Scale to reflectance

In [7]:
point = ee.Geometry.Rectangle([-118.87381926259474,33.676536492962484,-117.42191790156431,34.44541483548236])
#point = point_new.buffer(5000000)  # 5 km buffer around the point
sentinel_prefire = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(point).filterDate('2024-12-10','2025-01-06').sort('CLOUD_COVERAGE_ASSESSMENT').map(mask_clouds).median()
sentinel_postfire = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(point).filterDate('2025-01-07','2025-01-18').sort('CLOUD_COVERAGE_ASSESSMENT').map(mask_clouds).median()


#print(f"Pre-fire images available: {pre_sentinel}")
#print(f"Post-fire images available: {post_sentinel}")

In [8]:
# Debugging image availability
pre_sentinel_count = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(point) \
    .filterDate('2024-01-07', '2025-01-18') \
    .size() \
    .getInfo()

post_sentinel_count = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(point) \
    .filterDate('2024-12-10', '2025-01-06') \
    .size() \
    .getInfo()

print(f"Pre-fire images available: {pre_sentinel_count}")
print(f"Post-fire images available: {post_sentinel_count}")

Pre-fire images available: 461
Post-fire images available: 36


In [9]:
vis_par = {
    'min':0,
    'max':3000,
    'bands':['B4','B3','B2']
  }
map.addLayer(sentinel_prefire,vis_par, 'Sentinel_prefire')
map.addLayer(sentinel_postfire,vis_par, 'Sentinel_postfire')

In [10]:
#ndvi = image.normalizedDifference(['B8','B4'])

In [11]:
vis = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

In [12]:
pre = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(point).filterDate('2024-12-15','2025-01-06').sort('CLOUD_COVERAGE_ASSESSMENT').map(mask_clouds).median()#median() #.sort('CLOUD_COVERAGE_ASSESSMENT').first
post = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(point).filterDate('2025-01-07','2025-01-18').sort('CLOUD_COVERAGE_ASSESSMENT').map(mask_clouds).median()#median()

In [13]:
pre_burn = pre.reproject(crs='EPSG:4326', scale =10)
post_burn = post.reproject(crs='EPSG:4326', scale =10)
# 10 meters resolution
#pre_burn.getInfo()
#post_burn.getInfo()

In [14]:
B12 = pre_burn.select('B12')  # SWIR-2
B8a = pre_burn.select('B8A')  # Narrow NIR
B3 = pre_burn.select('B3')    # Green
B2 = pre_burn.select('B2')    # Blue

In [15]:
b12 = post_burn.select('B12')  # SWIR-2
b8a = post_burn.select('B8A')  # Narrow NIR
b3 = post_burn.select('B3')    # Green
b2 = post_burn.select('B2')    # Blue

In [16]:
num = B12.subtract(B8a).subtract(B3).subtract(B2)
denom = B12.add(B8a).add(B3).add(B2)

In [17]:
numerator = b12.subtract(b8a).subtract(b3).subtract(b2)
denominator = b12.add(b8a).add(b3).add(b2)

In [18]:
nbr_plus_pre = num.divide(denom).rename('nbr_plus_pre')
nbr_plus_post = numerator.divide(denominator).rename('nbr_plus_post')

In [19]:
print("NBR+ calculated:", nbr_plus_pre.getInfo())
print("NBR+ calculated:", nbr_plus_post.getInfo())

NBR+ calculated: {'type': 'Image', 'bands': [{'id': 'nbr_plus_pre', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 0, 0, -8.983152841195215e-05, 0]}]}
NBR+ calculated: {'type': 'Image', 'bands': [{'id': 'nbr_plus_post', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 0, 0, -8.983152841195215e-05, 0]}]}


In [20]:
net_nbr = nbr_plus_pre.subtract(nbr_plus_post).rename('net_nbr')
net_nbr.getInfo()


{'type': 'Image',
 'bands': [{'id': 'net_nbr',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    0,
    0,
    -8.983152841195215e-05,
    0]}]}

In [21]:
varis = {
    'min': -1,
    'max': 1,
    'palette': ['green', 'white', 'red']
}

In [22]:
map.addLayer(net_nbr, varis, 'Net NBR')
map.addLayer(nbr_plus_pre, varis, 'NBR+ pre')
map.addLayer(nbr_plus_post, varis, 'NBR+ post')

In [23]:
print(nbr_plus_post.bandNames().getInfo())
print(nbr_plus_pre.bandNames().getInfo())

['nbr_plus_post']
['nbr_plus_pre']


In [24]:
# Combine pre- and post-fire NBR into a single image
combined_image = nbr_plus_pre.addBands(nbr_plus_post)


In [25]:
#Define thresholds for burn severity
burn_severity = combined_image.expression(
    "b('nbr_plus_pre') - b('nbr_plus_post') > 0.66 ? 7 : "  # High severity
    "b('nbr_plus_pre') - b('nbr_plus_post') > 0.44 ? 6 : " # Moderate high severity
    "b('nbr_plus_pre') - b('nbr_plus_post') > 0.27 ? 5 : " # Moderate Low severity
    "b('nbr_plus_pre') - b('nbr_plus_post') > 0.10 ? 4 : " #Low
    "b('nbr_plus_pre') - b('nbr_plus_post') > -0.10 ? 3 : " #Unburned
    "b('nbr_plus_pre') - b('nbr_plus_post') > -0.25 ? 2 : " # Enhanced regrowth low
    "1",  # Enhanced regrowth low
    {
        'nbr_plus_pre': nbr_plus_pre,
        'nbr_plus_post': nbr_plus_post
    }
).rename('BurnSeverity')#

# Visualization parameters for burn severity
burn_vis = {
    'min': 1,
    'max': 7,
    'palette': ['green', 'lightgreen', 'yellow', 'orange', 'red', 'brown', 'black']
}

# Add the burn severity layer to the map
map.addLayer(burn_severity, burn_vis, 'Burn Severity')


In [26]:
# Define thresholds for burn severity
burn_severity = net_nbr.expression(
    "b('net_nbr') > 0.66 ? 7 : " +  # High severity
    "b('net_nbr') > 0.44 ? 6 : " +  # Moderate high severity
    "b('net_nbr') > 0.27 ? 5 : " +  # Moderate low severity
    "b('net_nbr') > 0.10 ? 4 : " +  # Low
    "b('net_nbr') > -0.10 ? 3 : " +  # Unburned
    "b('net_nbr') > -0.25 ? 2 : " +  # Enhanced regrowth low
    "1",  # Enhanced regrowth high
    {
        'net_nbr': net_nbr
    }
).rename('BurnSeverity')

# Visualization parameters for burn severity
burn_vis = {
    'min': 1,
    'max': 7,
    'palette': ['green', 'lightgreen', 'yellow', 'orange', 'red', 'brown', 'black']
}

# Add the burn severity layer to the map
map.addLayer(burn_severity, burn_vis, 'Burn Severity')  # Use 'burn_severity' here, not 'net_nbr'


In [27]:
# Calculate dNBR (Difference in NBR) from the pre- and post-fire NBR
dNBR = nbr_plus_pre.subtract(nbr_plus_post)
roi= ee.Geometry.Rectangle([-118.87381926259474,33.676536492962484,-117.42191790156431,34.44541483548236])
# Calculate the min and max values for dNBR
dNBR_min_max = dNBR.reduceRegion(
    reducer=ee.Reducer.minMax(),  # To get both min and max values
    geometry= roi,
    scale=30,                     # Set the scale (adjust based on the image's resolution)
    maxPixels=1e8                 # Optionally adjust to avoid data truncation for large images
)

# Print the min and max dNBR values
print(dNBR_min_max.getInfo())

{'nbr_plus_pre_max': 1.213277553627948, 'nbr_plus_pre_min': -1.5270072809304263}


In [28]:
map


Map(center=[34.172376, -118.288105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

calveg = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/LA Wildfire Assessment/Datasets (Light Files only)/LA Vegetation Datasets/Vegetation_CALVEG_Invasive/Current_InvasivePlants.shp")
calveg